In [1]:
import numpy as np
import random
import torch
import pandas as pd
import copy

from collections import Counter, defaultdict
import xml.etree.ElementTree as ET
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score


# World Sense Disambiguation (WSD)

Ce notebook permet d'effectuer une comparaison des performances de deux algorithmes, un d'apprentissage supervisé et un d'apprentissage semi-supervisé, pour la tâche de Word Sense Disambiguation (WSD). 

Les deux méthodes sont les suivants :
- pour la classification supervisée, utilisation d'un MLP
- pour la classification semi-supervisée, utilisation d'un algorithme de constrained K-Means

Plusieurs tests sont effectués en considérant plusieurs mots à désambiguiser pour lesquels les performances de ces deux méthodes sont évaluées. L'objectif final est de comparer ces performances pour déterminer à partir de combien de données annotées la première méthode est préférable à la deuxième et vice versa.



## Extraction des données

La classe Extractor permet d'accéder aux données et de les stocker dans des dictionnaires puis des embeddings. Elle permet de figer les données sur lesquelles les tests de classification seront ensuite faits. Elle comporte les méthodes suivantes :

- `extract_examples()` : pour extraire les données d'entraînement et de test provenant de fichiers XML FSE

- `extract_embeddings()` : pour extraire les embeddings à partir d'un fichier créé en amont, qui ne regroupe que les embeddings nécéssaires à notre jeu de données. Cette méthode est surtout utile pour drastiquement réduire le temps de chargement des embeddings

- `look_up()` : pour effectuer l'étape de look-up avant la classifcation

- `select_examples()` : pour sélectionner des données représentatives lorsqu'on ne considère pas toutes les données annotées. Pour qu'un ensemble de données soit représentatif, il faut que chaque étiquette présente dans les données totales soit présente au moins une fois dans le set d'entraînement sélectionné

- `define_instance()` : pour choisir les données qui seront classifiées, c'est-à-dire quelle proportion des données seront utilisées (80%, 50%, 25%...)


In [2]:
class Extractor :
    
    def __init__(self, data_path, gold_path, embeddings_path, context_size):
        '''
        A partir des données, extrait les examples qui seront utilisés lors de l'apprentissage.

        INPUT
        data_path (str) : chemin pour accéder aux données d'entraînement, au format XML
        gold_path (str) : chemin pour accéder aux numéros de sens correspondant aux données
                          d'entraînement
        embeddings_path (str) : chemin pour accéder aux embeddings, déjà extraits et 
                                présélectionnés pour ces données
        context_size (int) : taille de la fenêtre du contexte du mot à désambiguiser

        ARGUMENTS DE CLASSE
        self.w2examples (dict) : associe chaque lemme à une liste d'examples correspondant à son 
                                 utilisation dans une phrase du corpus. 
                                 examples (list) : liste de tous les couples 
                                  ([mots du contexte], numéro de sens) pour un lemme donné
        self.w2senses (dict) : associe chaque lemme à son ensemble de sens possible
        self.w2emb (dict) : associe chaque mot à son embedding
        self.selected_examples (liste) : liste d'examples qui contient au moins un example de 
                                         chaque sens du lemme choisi
        self.selected_examples_embs (liste) : même liste d'examples, mais avec les contextes 
                                              sous forme d'embedding
        self.X_train (set) : corpus d'apprentissage
        self.X_test (set) : numéros des sens correspondant au corpus d'apprentissage
        self.y_train (set) : corpus de test
        self.y_test (set) : numéros des sens correspondant au corpus d'apprentissage
        self.annotated_examples (list) : liste des données considérées comme annotées 
        self.unannotated_examples (list) : liste des données considérées comme non annotées 
        '''
        
        #récupération des données XML d'apprentissage fournies en argument
        tree = ET.parse(data_path)
        data_file = tree.getroot()

        #récupération des données txt correspondant aux gold class
        gold_file = open(gold_path, "r",encoding="utf-8")
        
        # self.w2examples (dict) : associe chaque lemme à une liste d'examples correspondant à son 
        #                          utilisation dans une phrase du corpus.
        # clé (str) : lemme du mot à désambiguiser
        # valeur (list) : liste de tous les couples ([mots du contexte], numéro de sens) pour ce 
        #                 lemme
        #
        # self.w2senses (dict) : associe chaque lemme à son ensemble de sens possible
        # clé (str) : lemme du mot à désambiguiser
        # valeur (set) : ensemble des sens possible pour ce lemme
        self.w2examples, self.w2senses = self.extract_examples_and_senses(data_file, gold_file, context_size)

        # self.w2emb (dict) : associe chaque mot à l'embedding représentant son contexte sommé
        # clé (str) : mot à désambiguiser
        # valeur (list) : embedding
        self.w2emb = self.extract_embeddings(embeddings_path)

        # liste d'examples qui contient au moins un example de chaque sens du lemme choisi
        # cette liste sera définie lorsque l'instance du mot à désambiguiser sera choisi
        self.selected_examples = []
        # même liste d'examples, mais avec les contextes sous forme d'embedding
        # cette liste sera définie lorsque l'instance du mot à désambiguiser sera choisi
        self.selected_examples_embs = []

        # les corpus seront définis lorsque l'instance du mot à désambiguiser sera choisi
        self.X_train = set()
        self.X_test = set()
        self.y_train = set()
        self.y_test = set()

        # liste des données considérées comme annotées 
        self.annotated_examples = []
        # liste des données considérées comme non annotées 
        self.unannotated_examples = []
    
    def extract_examples_and_senses(self, data_file, gold_file, context_size):
        '''
        Extrait les données à partir des fichiers de corpus d'apprentissage et de gold classes.

        INPUT
        data_file (Element): représentation des phrases du corpus d'apprentissage
        gold_file (TextIOWrapper): fichier contenant les numéros de sens (gold class) pour chaque
                                   mot à désambiguiser
        '''
    
        # clé (str) : lemme du mot à désambiguiser
        # valeur (list) : liste de tous les couples ([mots du contexte], numéro de sens) pour ce 
        #                 lemme
        w2examples = {}
        # clé (str) : lemme du mot à désambiguiser
        # valeur (set) : ensemble des sens possible pour ce lemme
        w2senses = defaultdict(set)

        # lecture du fichier gold
        gold_file = gold_file.readlines()
        
        # index de parcours dans le fichier gold
        i_gold = 0

        # pour chaque phrase du corpus
        for text_id in data_file:
            for sentence in text_id:

                # recherche de(s) l'indice(s) de(s) l'instance(s) pour savoir où se trouve la fenêtre
                i_instances = []
                for j in range(len(sentence)):
                    if sentence[j].tag == "instance":
                        i_instances.append(j)
                
                # tant qu'il y a des instances à repérer dans la phrase
                while len(i_instances) > 0:

                    # enregistrement du lemme de l'instance
                    instance = sentence[i_instances[0]].attrib["lemma"].lower()
                    
                    # si l'instance n'a pas encore de contexte
                    if instance not in w2examples : 
                        # le crée
                        w2examples[instance] = []

                    context = []
                    # pour chaque mot de la fenêtre
                    for k in range(-context_size, context_size + 1):
                        # si le mot est l'instance, enregistrement de son lemme
                        if k == 0:
                            context.append(instance)
                        # sinon enregistrement du mot du contexte
                        elif len(sentence) > i_instances[0] + k and i_instances[0] + k >= 0:
                            context.append(sentence[i_instances[0] + k].text.lower())
                    
                    # récupération des différents sens possibles dans le fichier gold 
                    gold_class = gold_file[i_gold].split("__")[1].split("_")[1]

                    # le fichiers gold et data ayant les données dans le même ordre, les instances
                    # et les sens peuvent être enregistrés en même temps dans leur dictionnaire
                    # respectif
                    w2senses[instance].add(gold_class)
                    w2examples[instance].append((context, gold_class))
                    
                    # l'instance lue est enlevée de la to-do list et l'index de parcours du fichier 
                    # gold est incrémenté pour passer à l'instance suivante
                    i_instances.pop(0)
                    i_gold += 1

        return w2examples, w2senses
    
    def extract_embeddings(self,path_embeddings) :
        '''
        Récupère les embeddings dans le fichier généré et associe à chaque mot son
        embedding.

        INPUT
        path_embeddings (str) : chemin pour accéder aux embeddings, déjà extraits et 
                                présélectionnés pour ces données

        OUTPUT
        w2emb (dict) : associe chaque mot à son embedding
        '''

        # lecture du fichier
        file_embeddings = open(path_embeddings , "r", encoding="UTF-8")

        # clé (str) : mot à désambiguiser
        # valeur (list) : embedding du mot
        w2emb = {}

        # pour chaque mot du fichier
        for line in file_embeddings.readlines():

            # séparation du mot et de l'embedding
            splitted_line = line.split(" ")
            word = splitted_line[0]
            embedding = list(map(float,splitted_line[1:]))
            # insertion du mot et de son embedding dans le dictionnaire
            w2emb[word] = embedding

        return w2emb

    def look_up(self, context) :
        '''
        Remplace dans les mots du vecteur de contexte par leur embedding et en fait la
        somme.

        INPUT
        context (list) : liste de taille (size_window*2)+1

        OUTPUT
        context_emb (list) : liste de taille size_embedding (300 ici)
        '''

        # taille d'un embedding : 300
        emb_size = len(list(self.w2emb.values())[0]) 
        # initialisation du contexte sous forme d'embedding
        context_emb = np.zeros(emb_size)

        # pour chaque mot du contexte
        for word in context :
            # s'il est présent dans le dictionnaire d'embeddings
            if word in self.w2emb :
                # il est ajouté à l'embedding représentant le contexte
                context_emb = np.add(context_emb, np.array(self.w2emb[word])) 

        return context_emb
    
    def select_examples(self, examples, senses, sample_size):
        '''
        Choisit des examples représentatifs du corpus selon un nombre imposé de 
        données à choisir.

        INPUT
        examples (list) : liste d'examples pour le mot à désambiguiser
        n_senses (int) : nombre de sens associés à ce mot
        sample_size (float) : quantité des données considérés

        OUTPUT
        selected_examples (list) : liste d'examples qui contient au moins un example 
                                   de chaque sens
        '''

        selected_examples = []
        
        # pour chaque sens, on ajoute un example associé à ce sens, au hasard
        for sense in senses :
            selected_examples.append(random.choice(list(filter((lambda example:example[1]==sense),
                                                               examples))))
        
        # ajoute le nombre de données manquantes (non-présentes dans la liste) selectionnées au hasard
        random.shuffle(examples)
        for example in examples:
            if example not in selected_examples and len(selected_examples) < sample_size*len(examples):
                selected_examples.append(example)

        return selected_examples
    
    def select_train_test(self, x, y, senses, train_size = 0.8):
        '''
        Sépare en un train set et test set les examples représentatifs du corpus en mettant 
        au moins une étiquette de chaque sens dans le train set (et le test set si possible)

        INPUT
        x (list) : liste d'examples pour le mot à désambiguiser
        y (list) : liste d'étiquette pour chaque example de x
        senses (list) : numéros des sens associés à ce mot
        train_size (float) : quantité des données d'entraînement considérés
        '''

        # SELECTION DES DONNEES REPRESENTATIVES DU TRAIN

        # sélection au hasard d'un contexte et de son étiquette pour chaque sens
        selected_train = [random.choice(list(filter((lambda example: example[1] == sense),
                                                    zip(x,y)))) for sense in senses]
            
        # SELECTION DES DONNEES REPRESENTATIVES DU TEST

        # fait de même pour le corpus de test, lorsque possible, en ne considérant pas
        # ceux déjà sélectionnés pour le train
        selected_test = []
        for sense in senses:
            added = False
            for example in zip(x,y):
                for train_ex in selected_train:
                    # vérifie que l'example n'a pas déjà été sélectionné dans le train et
                    # le test pour ne pas faire de doublon
                    if example[1] == sense and example[1] == train_ex[1] and not(np.array_equal(example[0], train_ex[0])):
                        selected_test.append(example)
                        added = True
                        break
                if added == True:
                    break

        # SELECTION DES DONNEES RESTANTES DU TRAIN ET DU TEST

        # effectue un split sur le reste d'examples disponibles
        remaining_X = []
        remaining_y = []
        selected_train_copy = copy.deepcopy(selected_train)
        selected_train_copy.extend(selected_test)
        for contexte, sens in zip(x,y):
            to_add = True
            for ex in selected_train_copy:
                # vérifie que l'example n'a pas déjà été sélectionné dans le train et le
                # test pour ne pas faire de doublon
                if np.array_equal(contexte, ex[0]) and sens == ex[1]:
                    to_add = False
                    break
            if to_add:
                remaining_X.append(contexte)
                remaining_y.append(sens)

        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(remaining_X, remaining_y, 
                                                                                train_size = train_size)
        
        # ajout des données représentatives dans les sets correspondant
        selected_X_train, selected_y_train = zip(*selected_train)
        selected_X_test, selected_y_test = zip(*selected_test)
        
        self.X_train.extend(selected_X_train)
        self.X_test.extend(selected_X_test)
        self.y_train.extend(selected_y_train)
        self.y_test.extend(selected_y_test)

            
    def define_instance(self, instance, data_size, affichage=True):
        '''
        Permet de définir l'instance du mot à désambiguiser et les examples à utiliser. 
        Fixe les derniers arguments de classe selon l'instance choisie.
        Contraint le choix de classification selon le nombre de données considéré.

        INPUT
        instance (str) : mot à désambiguiser
        data_size (float): quantité de données à considérer
        affichage (bool, optional): affiche une trace des options choisies
        '''

        # choisit des examples d'entraînement représentatifs du corpus selon le nombre
        # imposé de données
        self.annotated_examples = self.select_examples(self.w2examples[instance],
                                                       self.w2senses[instance],
                                                       data_size)
        
        # garde en mémoire les examples considérés comme non-annotés
        self.unannotated_examples = [example for example in self.w2examples[instance] 
                                     if example not in self.annotated_examples]
        # liste de tous les contextes sous forme d'embedding
        x = [self.look_up(context) for context, gold in self.annotated_examples]
        # liste de tous les numéros de sens
        y = [gold for context, gold in self.annotated_examples]
        
        # liste des examples d'entrainement représentatifs sous forme d'embeddings
        self.selected_examples_embs = list(zip(x,y))
        
        # création des corpus d'entrainement et de test
        self.select_train_test(x, y, self.w2senses[instance])
        
        # si une trace est souhaitée
        if affichage :
            print("instance :", instance)
            print(f'{data_size*100}% des données annotées considérées')
            print("nombre de données d'entraînement : ", len(self.X_train))
            print("étiquettes possibles pour cette instance : ", self.w2senses[instance])
            print("étiquettes présentes dans les données d'entraînement :", Counter(self.y_train))
        return
    
    def get_sets(self):
        return self.X_train, self.X_test, self.y_train, self.y_test
    
    def get_annotated_examples(self):
        return self.annotated_examples
    
    def get_unannotated_examples(self):
        return self.unannotated_examples
    
    def get_embs(self):
        return self.selected_examples_embs

In [3]:
# chemin du corpus annoté
data_path = "../donnees/FSE-1.1-191210/FSE-1.1.data.xml"
# chemin pour récupérer les gold class du corpus annoté
gold_path = "../donnees/FSE-1.1-191210/FSE-1.1.gold.key.txt"
# choix de la fenêtre du contexte
context_size = 4
# chemin pour récupérer les embeddings afin de faire l'opération look-up
# Les embeddings sont extraits de fasttext
embeddings_path = "embeddings.txt"

In [4]:
# exemple d'utilisation d'Extractor avec le verbe "aboutir" et 100% des données utilisées
ext = Extractor(data_path, gold_path, embeddings_path, context_size)
ext.define_instance("aboutir", 1)
print("\nexemple d'example avec contexte et gold class :",ext.get_annotated_examples()[0])
print("\nau format embedding : ",ext.get_embs()[0])

instance : aboutir
100% des données annotées considérées
nombre de données d'entraînement :  39
étiquettes possibles pour cette instance :  {'2', '1', '4', '3'}
étiquettes présentes dans les données d'entraînement : Counter({'3': 31, '4': 6, '2': 1, '1': 1})

exemple d'example avec contexte et gold class : (['les', 'produits', 'ainsi', 'formés', 'aboutir', 'dans', 'des', 'vésicules', 'destinées'], '2')

au format embedding :  (array([ 8.6700e-02, -8.6500e-02,  1.1530e-01, -1.9800e-02, -5.4330e-01,
       -1.1660e-01,  4.0650e-01, -3.2600e-01,  1.4230e-01, -9.7700e-02,
        1.2140e-01, -3.3560e-01,  1.1430e-01,  1.2610e-01,  4.3000e-02,
        6.0600e-02,  8.3600e-02, -4.5000e-02,  2.9500e-02,  9.6500e-02,
       -3.4000e-03, -1.3260e-01, -7.7800e-02, -2.4500e-02, -7.0000e-04,
        1.6780e-01, -8.2870e-01, -1.3090e-01,  5.3910e-01, -1.6000e-03,
       -1.1190e-01,  9.0000e-04, -3.9980e-01,  3.1190e-01,  1.2870e-01,
        1.8070e-01,  5.4570e-01, -4.8000e-03, -1.1460e-01, -6.257


## Classification supervisée

L'utilisation d'un Classifieur MLP permet de prédire le sens des verbes de façon supervisée. Il s'agit de la première méthode proposée pour la tâche de WSD.

Les données utilisées sont les données annotées extraites du fichier FSE grâce à l'Extractor pour un lemme donné. Un MLPClassifier_WSD doit donc être créé pour chaque verbe dont on veut la prédiction.

MLPClassifier_WSD comporte les méthodes suivantes :
- `classify()` : pour prédire le sens du lemme associé au classifieur et calculer l'accuracy de cette prédiction


In [5]:
class MLPClassifier_WSD :
    
    def __init__(self, X_train, X_test, y_train, y_test) :
        '''
        A partir des données, prédit le sens d'un lemme et évalue la prédiction

        ARGUMENTS DE CLASSE
        self.X_train (list) : liste des contextes d'entraînement
        self.y_train (list) : liste des gold class d'entraînement
        self.X_test (list) : liste des contextes de test
        self.y_test (list) : liste des gold class de test
        '''
        # ensembles d'entraînement et de test provenant de l'Extractor pour un lemme donné
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test 

        # prédiction du modèle
        self.y_pred = []
        # accuracy de la prédiction
        self.accuracy = 0
        
    def classify(self, affichage=False) :
        ''' 
        Permet de prédire le sens du verbe.

        INPUT
        affichage (bool, optional) : affiche une trace des options choisies
        '''

        clf = MLPClassifier(random_state=1, hidden_layer_sizes=(100,)) 
        # entraînement
        clf.fit(self.X_train, self.y_train)
        # prédiction
        self.y_pred = clf.predict(self.X_test)

        # calcul de l'accuracy
        self.accuracy = accuracy_score(self.y_pred, self.y_test)
        
        # si une trace est souhaitée
        if affichage :
            print("prediction :", self.y_pred)
            print("gold :", self.y_test)
            print("accuracy score : ", self.accuracy,"\n")

In [6]:
# exemple d'utilisation de MLPClassifieur_WSD avec le verbe "demeurer" et 100% des données utilisées
ext = Extractor(data_path, gold_path, embeddings_path, context_size)
ext.define_instance("demeurer", 1, affichage=False)
X_train, X_test, y_train, y_test = ext.get_sets()

Clf = MLPClassifier_WSD(X_train, X_test, y_train, y_test)
Clf.classify(affichage=True)

prediction : ['4' '4' '4' '2' '4' '4' '4' '4' '4' '4' '4']
gold : ['4', '4', '4', '4', '4', '4', '4', '4', '3', '4', '2']
accuracy score :  0.7272727272727273 



/home/mathilde/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Pour pouvoir comparer les performances de ce Classifieur MLP avec celles de l'apprentissage semi-supervisé, nous devons créer un classifieur par verbe à désambiguiser et choisir un pas de descente pour la quantité de données annotées considérées.

Pour chaque classifieur et chaque quantitée de données considérées, on effectue *n_repeat* classifications pour obtenir une accuracy moyenne représentative du classifieur. 

Par conséquent, pour :
- n_repeat = 5
- step = 0.25

nous obtiendrons pour chaque classifieur une liste d'accuracies correspondant à :
- la moyenne des accuracies de 5 prédictions 
- pour 100%, 75%, 50% et 25% des données.

In [7]:
def get_mean_accuracy(instance, data_size, n_repeat, affichage=True):
    '''
    Retourne une moyenne d'accuracies pour un nombre de classifications donné.

    INPUT
    n_repeat (int): nombre de classifications à effectuer
    affichage (bool, optional): affiche une trace des options choisies

    OUTPUT
    mean (float): moyenne des accuracies calculées
    '''

    # liste de l'accuracy à chaque répétition
    accuracies = []
        
    for _ in range(n_repeat) :
            
        # pour chaque itération, un nouveau classifieur est créé, qui s'appuie toujours sur 
        # la même quantité de données mais avec un autre tirage des données
        # par exemple, pour 75% des données, des accuracies différentes seront obtenues en
        # fonction des tirages de ces données
        ext.define_instance(instance, data_size, False)
        X_train, X_test, y_train, y_test = ext.get_sets()
        Clf = MLPClassifier_WSD(X_train, X_test, y_train, y_test)
        Clf.classify()
            
        accuracies.append(Clf.accuracy)
        mean = sum(accuracies)/len(accuracies)
            
    # si la trace est souhaitée
    if affichage :
        print(f'{data_size*100}% des données annotées considérées')
        print("instance :", instance)
        print("accuracies :", accuracies)
        print("accuracy moyenne :", mean, "\n")

    return mean

def get_accuracies(instances, step, n_repeat, affichage=True):
    '''
    - Permet d'obtenir une accuracy moyenne par lemme pour une certaine quantité 
      de données considérée.
    - Produit un fichier csv.

    INPUT
    instances (list): liste des mots à désambiguiser
    step (float): pas de descente dans la quantité de données à considérer
    n_repeat (int): nombre de classifications tests à effectuer

    OUTPUT
    instance2acc (dict) : associe à chaque instance sa liste d'accuracies moyennes
    '''

    # clé : mot à désambiguiser
    # valeur : liste des accuracies moyennes à chaque pas de descente
    instance2acc = {instance : [] for instance in instances}
    moyennes = []
    data_sizes =[]
            
    # pour chaque quantité de données observée
    for i in range(round(1.0/step)): 
                
        # calcul de la taille des données avec le pas de descente (100%, 75%, 50%, 25%)
        data_size = 1.0 - (step*float(i))
        data_sizes.append(data_size)
        
        # pour chaque mot à désambiguiser
        for instance in instances :
            # calcul de son accuracy moyenne
            instance2acc[instance].append(get_mean_accuracy(instance, data_size, n_repeat, affichage))

        # on boucle sur les valeurs obtenues pour obtenir leur moyenne
        moyennes.append(sum([accuracies[i] for accuracies in instance2acc.values()])/len(instance2acc))
  
    # ajout d'une clé moyennes pour avoir les moyennes de toutes les accuracy moyennes pour chaque instance
    # pour chaque pas
    instance2acc["moyennes"] = moyennes
            
    # ajout d'une clé pour stocker les tailles des données à chaque pas
    instance2acc["data_sizes"] = data_sizes 

    # export en csv des résultats
    df = pd.DataFrame(instance2acc)
    df.set_index("data_sizes")
    df.to_csv(f"accuracies_par_lemme_{n_repeat}_repet.csv")
    
    return instance2acc     

In [8]:
# pas de descente dans la quantité de données considérées
step = 0.25
# nombre de classifications pour un classifieur pour obtenir une accuracy moyenne
n_repeat = 5

In [9]:
# récupération de tous les verbes du corpus à désambiguiser
instances = list(ext.w2examples.keys())
# calcul des accuracies
get_accuracies(instances, step, n_repeat)

100.0% des données annotées considérées
instance : aboutir
accuracies : [0.8181818181818182, 0.8181818181818182, 0.9090909090909091, 1.0, 0.8181818181818182]
accuracy moyenne : 0.8727272727272727 

100.0% des données annotées considérées
instance : investir
accuracies : [0.6666666666666666, 0.75, 0.5833333333333334, 0.5, 0.5]
accuracy moyenne : 0.6 

100.0% des données annotées considérées
instance : traduire
accuracies : [0.5833333333333334, 0.75, 0.75, 0.75, 0.75]
accuracy moyenne : 0.7166666666666667 

100.0% des données annotées considérées
instance : témoigner
accuracies : [0.8333333333333334, 0.75, 0.8333333333333334, 0.9166666666666666, 0.75]
accuracy moyenne : 0.8166666666666668 

100.0% des données annotées considérées
instance : juger
accuracies : [0.6, 0.7, 0.6, 0.6, 0.5]
accuracy moyenne : 0.6 



/home/mathilde/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/mathilde/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/mathilde/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/mathilde/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/mathilde/.local/lib/python3.10/sit

100.0% des données annotées considérées
instance : justifier
accuracies : [0.7272727272727273, 0.7272727272727273, 0.9090909090909091, 0.7272727272727273, 0.8181818181818182]
accuracy moyenne : 0.7818181818181819 

100.0% des données annotées considérées
instance : viser
accuracies : [0.9166666666666666, 1.0, 1.0, 1.0, 1.0]
accuracy moyenne : 0.9833333333333332 



/home/mathilde/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/mathilde/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/mathilde/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/mathilde/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/mathilde/.local/lib/python3.10/sit

100.0% des données annotées considérées
instance : prononcer
accuracies : [0.6153846153846154, 0.46153846153846156, 0.6153846153846154, 0.8461538461538461, 0.5384615384615384]
accuracy moyenne : 0.6153846153846154 

100.0% des données annotées considérées
instance : accomplir
accuracies : [0.6923076923076923, 0.6923076923076923, 0.7692307692307693, 0.6923076923076923, 0.6923076923076923]
accuracy moyenne : 0.7076923076923076 



...

/home/mathilde/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


50.0% des données annotées considérées
instance : appartenir
accuracies : [0.7142857142857143, 0.5714285714285714, 0.7142857142857143, 0.16666666666666666, 0.2857142857142857]
accuracy moyenne : 0.4904761904761904 

50.0% des données annotées considérées
instance : arriver
accuracies : [0.625, 0.5714285714285714, 0.375, 0.375, 0.375]
accuracy moyenne : 0.46428571428571425 

50.0% des données annotées considérées
instance : chercher
accuracies : [0.7142857142857143, 0.8571428571428571, 0.8571428571428571, 0.5714285714285714, 0.5]
accuracy moyenne : 0.7 

50.0% des données annotées considérées
instance : composer
accuracies : [1.0, 1.0, 1.0, 1.0, 1.0]
accuracy moyenne : 1.0 

50.0% des données annotées considérées
instance : confier
accuracies : [0.7142857142857143, 0.8571428571428571, 0.5714285714285714, 0.8571428571428571, 1.0]
accuracy moyenne : 0.8 

25.0% des données annotées considérées
instance : aboutir
accuracies : [0.75, 0.75, 0.75, 1.0, 1.0]
accuracy moyenne : 0.85 

25.0% des

/home/mathilde/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


25.0% des données annotées considérées
instance : justifier
accuracies : [0.25, 0.75, 1.0, 0.5, 0.6666666666666666]
accuracy moyenne : 0.6333333333333333 

25.0% des données annotées considérées
instance : viser
accuracies : [1.0, 1.0, 1.0, 0.75, 0.75]
accuracy moyenne : 0.9 

25.0% des données annotées considérées
instance : prononcer
accuracies : [0.5, 0.3333333333333333, 0.4, 0.3333333333333333, 0.6]
accuracy moyenne : 0.4333333333333333 

25.0% des données annotées considérées
instance : accomplir
accuracies : [0.25, 0.75, 0.5, 0.75, 1.0]
accuracy moyenne : 0.65 

25.0% des données annotées considérées
instance : convenir
accuracies : [0.5, 0.4, 0.5, 0.4, 0.6]
accuracy moyenne : 0.48 

25.0% des données annotées considérées
instance : acquérir
accuracies : [0.25, 0.75, 0.5, 1.0, 0.5]
accuracy moyenne : 0.6 

25.0% des données annotées considérées
instance : achever
accuracies : [0.5, 0.5, 0.75, 0.75, 0.75]
accuracy moyenne : 0.65 

25.0% des données annotées considérées
instance : 

ValueError: With n_samples=1, test_size=None and train_size=0.8, the resulting train set will be empty. Adjust any of the aforementioned parameters.

## Classification semi-supervisée

In [ ]:
class K_Means():
    ''' 
    classifieur K-means pour un mot particulier
    '''

    def __init__(self, examples):
        '''
        Instancie les différentes variables utiles pour l'algorithme du K-means

        examples : liste d'examples dont le mot à désambiguiser est le même pour 
                   chaque example
        example : couple d'un mot avec son contexte de fenêtre 4 (sous forme 
                  d'embedding) et du numéro de sens attendu du mot à désambiguiser 
                  (gold class sous forme d'integer)
        '''

        # transforme l'ensemble des examples en une liste pour pouvoir garder le 
        # même indice pour chaque example par la suite
        self.examples = list(examples)
        # transforme les embeddings en tensors
        self.tensors_examples = [example[0] for example in self.examples]
        # détermine le nombre de sens possibles k (donc le nombre de clusters) 
        # à l'aide des données
        self.k = self.nb_senses()
        # initialisation de centroids : pour chaque sens, un example est pris au hasard
        self.tensors_centroids = [random.choice(example) 
                                  for example in self.examples_of_same_sense().values()]
        # initialisation de clusters : tous les examples sont associés au cluster 0
        self.clusters = np.zeros(len(examples))

    def nb_senses(self):
        '''
        Renvoie le nombre de sens existants dans un ensemble d'examples
        '''

        known_senses = []
        # pour chaque exemple
        for example in self.examples:
            # si le sens attendu (gold class) n'a pas encore été rencontré
            if example[1] not in known_senses:
                # l'ajoute à la liste des sens possibles
                known_senses.append(example[1])
        # renvoie le nombre de sens
        return len(known_senses)
    
    def examples_of_same_sense(self):
        '''
        Regroupe les contextes des examples dans un dictionnaire triés selon le 
        sens du mot à désambiguiser
        '''

        # clé : numéro du sens
        # valeur : liste de contextes avec ce sens en gold class
        sense2examples = {}
        # pour chaque example
        for example in self.examples:
            # si sa gold class n'a pas été déjà rencontrée
            if example[1] not in sense2examples:
                # ajoute une clé pour cette gold class
                sense2examples[example[1]] = []
            # ajoute le contexte au dictionnaire correspondant au sens utilisé
            sense2examples[example[1]].append(example[0])

        return sense2examples
    
    def learn_clusters(self):
        '''
        Algorithme de K-Means
        Retourne les coordonnées de chaque centroide ainsi que le cluster auquel 
        appartient chaque example
        '''

        # différence initialisée à Vrai
        diff = True
        
        # tant qu'il y a une différence entre l'ancienne liste et la nouvelle 
        # liste de centroides
        while diff:

            # CALCUL DES DISTANCES ENTRE CHAQUE EXAMPLE ET CHAQUE CENTROIDE

            # pour chaque couple (indice, coordonnées) dans les examples
            for i, tensor_example in enumerate(self.tensors_examples):
                # initialisation de la distance minimum à l'infini
                min_dist = float('inf')
                # pour chaque couple (indice, coordonnées) dans les centroides
                for j, tensor_centroid in enumerate(self.tensors_centroids):
                    # calcul de la distance entre cet example et ce centroide
                    d = 0
                    for k in range(len(tensor_example)):
                        d += (tensor_centroid[k].item() - tensor_example[k].item())**2
                    d = np.sqrt(d)
                    # si une distance plus faible est trouvée
                    if min_dist > d:
                        # la distance ainsi que le centroide sont stockés
                        min_dist = d
                        self.clusters[i] = j
            
            # CALCUL DES NOUVEAUX CENTROIDES

            # calcul des nouveaux centroides en utilisant le point au milieu de tous les
            # autres points du même cluster
            new_centroids = pd.DataFrame(self.tensors_examples).groupby(by = self.clusters).mean()
            # transforme ces nouveaux centroides en tensors
            tensors_new_centroids = []
            for i in range(len(new_centroids.index)):
                colums = []
                for j in range(len(new_centroids.columns)):
                    colums.append(int(new_centroids.iat[i,j]))
                tensors_new_centroids.append(torch.tensor(colums))

            # MISE A JOUR DES CENTROIDES

            count_diff = 0
            # pour chaque centroide
            for i in range(len(self.tensors_centroids)):
                # si l'ancien centroide et le nouveau ne sont pas les mêmes
                if not(torch.equal(self.tensors_centroids[i], tensors_new_centroids[i])):
                    count_diff += 1
                    # met à jour le centroide
                    self.tensors_centroids = tensors_new_centroids
            # s'il n'y a eu aucune différence entre les anciens et les nouveaux centroides, 
            # la boucle while se termine
            if count_diff == 0:
                diff = False